In [17]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, time
from termcolor import colored

## Part _: _

In [18]:
# Read in the results from model predictions
TESTDATADIR = "../data/test_stocks/"
TESTSTOCKS = [
    "AAPL", # Apple
    "AMD",  # AMD
    "AMZN", # Amazon
    "META", # Meta Platforms
    "NFLX", # Netflix
    "QCOM", # Qualcomm
    "SBUX", # Starbucks
    "SCSO", # Cisco
    "TSLA", # Tesla
]
predictions = {}
for stock in TESTSTOCKS:
    predictions[stock] = pd.read_parquet(TESTDATADIR + stock + "_results.parquet")

# Functionality for simulating trades

In [19]:
# Define the buy signal logic
def buy_shares(cash, actual_price_today, fraction=1.0):
    """
    Buys as many shares as possible with the fraction of available cash.
    Returns the updated cash and number of shares bought.
    """
    shares = int(cash * fraction / actual_price_today)
    cash -= shares * actual_price_today
    return cash, shares

# Define the sell signal logic
def sell_shares(cash, shares, actual_price_today, fraction=1.0):
    """
    Sells a fraction of the shares owned.
    Returns the updated cash and sets shares to new value.
    """
    shares_to_sell = int(shares * fraction)
    cash += shares_to_sell * actual_price_today
    shares -= shares_to_sell
    return cash, shares

In [20]:
def simulate_trading_strategy(
    actual: np.ndarray,
    predicted: np.ndarray,
    action: callable,
    initial_cash: float = 10000.0
) -> tuple:
    """
    Simulates a trading strategy based on the actual and predicted stock prices.
    The action function should define the buy/sell logic. Which takes the current
    cash, actual price, and predicted price. The function returns a tuple 
    containing the updated cash and number of shares and portfolio value over 
    each iteration. The action function should return a positive value to buy,
    negative value to sell, and zero to do nothing. These abs(result) of the action
    function will be passed to the buy/sell function as fraction of stock to
    sell or fraction of cash used to buy.
    """
    # Initialize variables
    cash = initial_cash
    shares = [0]
    portfolio_value = [initial_cash]
    
    # Iterate through the actual and predicted prices
    for actual_price_today, predicted_price_today in zip(actual, predicted):
        # Buy shares if the predicted price is higher than the actual price
        action_value = action(cash, actual_price_today, predicted_price_today)
        
        if action_value > 0: # Buy shares
            cash, shares_today = buy_shares(cash, actual_price_today, abs(action_value))
            shares.append(shares_today)
        elif action_value < 0: # Sell shares
            cash, shares_today = sell_shares(cash, shares[-1], actual_price_today, abs(action_value))
            shares.append(shares_today)
        else: # No action
            shares.append(shares[-1])
            
        # Calculate the the portfolio value after each iteration
        portfolio_value.append(cash + shares[-1] * actual_price_today)
    
    return cash, shares, portfolio_value

In [21]:
# This uses a random action function to decide whether to buy or sell shares
def random_action(*args, **kwargs):
    """
    Randomly decides to buy or sell shares.
    """
    return np.random.choice([.5, 1, -.5])

# Use the predicted stock to determine value to determine the action.
def prediction_action(actual, predicted):
    """
    Decides to buy or sell shares based on the predicted price.
    """
    if predicted > actual:
        return .5 # Buy with half of the cash
    elif actual > predicted:
        return -.5 # Sell half of the shares
    else:
        return 0 # No action

## Part _: Random Trading

In [22]:
# Random strategy simulation store
random_results = {}

# Simulate the trading strategy for each stock.
for ticker, results in predictions.items():
    
    cash, shares, portfolio_value = simulate_trading_strategy(
        np.array(results['Actual'].values),
        np.array(results['Prediction'].values),
        random_action,
        initial_cash=10000.0
    )
    
    # Create a DataFrame to hold the results
    data = pd.DataFrame({
        'Date': results['Date'],
        'Actual': results['Actual'],
        'Prediction': results['Prediction'],
        'Shares': shares[1:],
        'Portfolio Value': portfolio_value[1:],
    })
    
    # Add the final cash value to the DataFrame
    random_results[ticker] = data

## Part _: Trading with predictions

In [23]:
# Simulate paper trading for each stock
def simulate_paper_trading(predictions, 
                           initial_investment=10000):
    """
    Simulates paper trading for each stock in the predictions dictionary.
    Returns a DataFrame with the final portfolio value and yield for each stock.
    """
    trading_results = []

    for ticker, results in predictions.items():
        cash = initial_investment
        shares = 0
        portfolio_value = []

        # Iterate through the results DataFrame
        for i in range(len(results) - 1):  # Exclude the last day since we can't predict beyond it
            actual_price_today = results['Actual'].iloc[i]
            predicted_price_tomorrow = results['Prediction'].iloc[i + 1]

            # Buy signal: Predicted price tomorrow > Actual price today
            if predicted_price_tomorrow > actual_price_today:
                cash, shares_bought = buy_shares(cash, actual_price_today)
                shares += shares_bought

            # Sell signal: Predicted price tomorrow < Actual price today
            elif predicted_price_tomorrow < actual_price_today and shares > 0:
                cash, shares = sell_shares(cash, shares, actual_price_today)

            # Calculate the portfolio value (cash + value of shares)
            portfolio_value.append(cash + shares * actual_price_today)

        # Final portfolio value
        final_portfolio_value = cash + shares * results['Actual'].iloc[-1]
        yield_percentage = ((final_portfolio_value - initial_investment) / initial_investment) * 100

        # Store the results
        trading_results.append({
            'Ticker': ticker,
            'Final Portfolio Value': final_portfolio_value,
            'Yield (%)': yield_percentage
        })

        # Print results for the stock
        print(f"Results for {ticker}:")
        print(f"   Final Portfolio Value: ${final_portfolio_value:.2f}")
        print(f"   Yield: {yield_percentage:.2f}%")
        print()

    # Convert trading results to a DataFrame for display
    trading_results_df = pd.DataFrame(trading_results)
    return trading_results_df

# Run the simulation
trading_results_df = simulate_paper_trading(predictions, initial_investment=10000)

# Display the trading results
display(trading_results_df)

Results for AAPL:
   Final Portfolio Value: $9159.59
   Yield: -8.40%

Results for AMD:
   Final Portfolio Value: $9991.00
   Yield: -0.09%

Results for AMZN:
   Final Portfolio Value: $11619.27
   Yield: 16.19%

Results for META:
   Final Portfolio Value: $8932.57
   Yield: -10.67%

Results for NFLX:
   Final Portfolio Value: $8007.13
   Yield: -19.93%

Results for QCOM:
   Final Portfolio Value: $12167.89
   Yield: 21.68%

Results for SBUX:
   Final Portfolio Value: $9629.54
   Yield: -3.70%

Results for SCSO:
   Final Portfolio Value: $9347.09
   Yield: -6.53%

Results for TSLA:
   Final Portfolio Value: $8581.51
   Yield: -14.18%



,Ticker,Final Portfolio Value,Yield (%)
0,AAPL,9159.59,-8.4041
1,AMD,9991.00,-0.0900
2,AMZN,11619.27,16.1927
3,META,8932.57,-10.6743
4,NFLX,8007.13,-19.9287
5,QCOM,12167.89,21.6789
6,SBUX,9629.54,-3.7046
7,SCSO,9347.09,-6.5291
8,TSLA,8581.51,-14.1849


In [24]:
# Print a header
print("\n{:<8} {:<22} {:<10}".format('Ticker', 'Final Portfolio Value', 'Yield (%)'))
print("-" * 45)

# Iterate through the test data and print colored results
for ticker, results in predictions.items():
    final_value = trading_results_df.loc[trading_results_df['Ticker'] == ticker, 'Final Portfolio Value'].values[0]
    yield_pct = trading_results_df.loc[trading_results_df['Ticker'] == ticker, 'Yield (%)'].values[0]
    
    # Format the values
    final_value_str = "${:.2f}".format(final_value)
    yield_str = "{:.2f}%".format(yield_pct)
    
    # Color based on yield (green for positive, red for negative)
    if yield_pct >= 0:
        colored_yield = colored(yield_str, 'green')
    else:
        colored_yield = colored(yield_str, 'red')
    
    # Print the row
    print("{:<8} {:<22} {}".format(ticker, final_value_str, colored_yield))


Ticker   Final Portfolio Value  Yield (%) 
---------------------------------------------
AAPL     $9159.59               -8.40%
AMD      $9991.00               -0.09%
AMZN     $11619.27              16.19%
META     $8932.57               -10.67%
NFLX     $8007.13               -19.93%
QCOM     $12167.89              21.68%
SBUX     $9629.54               -3.70%
SCSO     $9347.09               -6.53%
TSLA     $8581.51               -14.18%


In [26]:
  # Create a dataframe from the results.
dataframe_columns = ['Ticker', 'Actual', 'Prediction', 'Date']
trading_results_df = pd.DataFrame(columns=dataframe_columns)
for ticker, results in predictions.items():
    # Create a DataFrame for the results
    results_df = pd.DataFrame({
        'Ticker': ticker,
        'Actual': results['Actual_Scaled'],
        'Prediction': results['Prediction_Scaled'],
        'Date': results['Date']
    })
    
    # Append to the main DataFrame
    trading_results_df = pd.concat([trading_results_df, results_df], ignore_index=True)
    
# Display the trading results DataFrame
display(trading_results_df)

/tmp/ipykernel_59940/1302842159.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trading_results_df = pd.concat([trading_results_df, results_df], ignore_index=True)


,Ticker,Actual,Prediction,Date
0,AAPL,0.592725,0.532328,2025-03-25
1,AAPL,0.557852,0.553596,2025-03-24
2,AAPL,0.529446,0.521709,2025-03-21
3,AAPL,0.481293,0.500552,2025-03-20
4,AAPL,0.494457,0.464448,2025-03-19
...,...,...,...,...
1975,TSLA,0.029160,0.206345,2024-05-14
1976,TSLA,0.010983,0.209055,2024-05-13
1977,TSLA,0.000000,0.204308,2024-05-10
1978,TSLA,0.011240,0.200990,2024-05-09


## Part _: Trading with predictions and market sentiment.